In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [2]:
from qml_hep_lhc.data import MNIST
from qml_hep_lhc.layers.utils import convolution_iters
import argparse

2022-08-07 08:25:16.631382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-07 08:25:16.631481: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-07 08:26:32.417535: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-07 08:26:32.417598: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-08-07 08:26:32.438770: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [296]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.labels_to_categorical = 1
args.processed = 1
args.percent_samples = 0.1

# Base Model
args.optimizer = 'Ranger'
args.batch_size = 4
args.epochs = 3

# Quantum CNN Parameters
n_layers = 1
n_qubits = 9
template = 'S2D'

In [297]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤═══════════════╤═══════════════╤═══════════════╤═══════════╕
│ Data   │ Train size    │ Val size      │ Test size     │ Dims      │
╞════════╪═══════════════╪═══════════════╪═══════════════╪═══════════╡
│ X      │ (48, 8, 8, 1) │ (12, 8, 8, 1) │ (20, 8, 8, 1) │ (8, 8, 1) │
├────────┼───────────────┼───────────────┼───────────────┼───────────┤
│ y      │ (48, 2)       │ (12, 2)       │ (20, 2)       │ (2,)      │
╘════════╧═══════════════╧═══════════════╧═══════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.58 │  7.68 │  -0.01 │  0.98 │ [

In [298]:
import tensorflow as tf
import numpy as np

tf.keras.backend.set_floatx('float64')

In [299]:
import pennylane as qml


dev = qml.device("lightning.qubit", wires=n_qubits)
qubits = list(range(n_qubits))
n_inputs = 9

In [300]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_sparse(inputs, weights, bias):
    z = tf.tensordot(weights, inputs, axes = 1) + bias
    for l in range(n_layers):
        for q in qubits:
            qml.Rot(z[l,q,0], z[l,q,1], z[l,q,2], wires= q)
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))         
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [301]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_basic(inputs, weights, bias):
    inputs = inputs + bias
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [302]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_s2d(inputs, weights, bias):
    z = tf.tensordot(weights, inputs, axes = 1) + bias
    qml.SimplifiedTwoDesign(initial_layer_weights=tf.zeros((n_qubits,)), weights=z, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [303]:
def get_node(template, num_layers, num_qubits, num_inputs):
    if template == 'NQubitPQC':
        return qnode_sparse, {
            "weights": (num_layers, num_qubits, 3, num_inputs),
            "bias": (num_layers, num_qubits, 3)
        }
    elif template == 'Basic':
        assert num_inputs == n_qubits
        return qnode_basic, {
            "weights": (num_layers, num_qubits),
            "bias": (num_qubits,)
        }
    elif template == 'S2D':
        return qnode_s2d, {
            "weights": (num_layers, num_qubits -1, 2, num_inputs),
            "bias": (num_layers, num_qubits -1, 2)
        }

In [304]:
inputs = tf.random.uniform((n_qubits, ))
node, shapes = get_node(template, n_layers, n_qubits, n_inputs)
w = tf.random.uniform(shapes['weights'])
b = tf.random.uniform(shapes['bias'])
drawer = qml.draw(node, expansion_strategy="device")
print(drawer(inputs,w,b))

0: ──RY(0.00)─╭●──RY(1.56)──────────────┤  <Z>
1: ──RY(0.00)─╰Z──RY(1.88)─╭●──RY(1.96)─┤  <Z>
2: ──RY(0.00)─╭●──RY(2.06)─╰Z──RY(2.00)─┤  <Z>
3: ──RY(0.00)─╰Z──RY(1.97)─╭●──RY(1.39)─┤  <Z>
4: ──RY(0.00)─╭●──RY(1.26)─╰Z──RY(2.93)─┤  <Z>
5: ──RY(0.00)─╰Z──RY(1.93)─╭●──RY(2.21)─┤  <Z>
6: ──RY(0.00)─╭●──RY(1.36)─╰Z──RY(2.61)─┤  <Z>
7: ──RY(0.00)─╰Z──RY(1.88)─╭●──RY(2.02)─┤  <Z>
8: ──RY(0.00)──────────────╰Z──RY(2.13)─┤  <Z>


In [305]:
class QConv2D(tf.keras.layers.Layer):
    """
    2D Quantum convolution layer (e.g. spatial convolution over images).
    This layer creates a convolution kernel that is convolved 
    with the layer input to produce a tensor of outputs. Finally,
    `activation` is applied to the outputs as well.
    """

    def __init__(
            self,
            template = 'Basic',
            name='QConv2D',
    ):

        super(QConv2D, self).__init__(name=name)

        self.kernel_size = (3,3)
        self.strides = (2,2)
        
        node, shapes = get_node(template, n_layers, n_qubits, np.prod(self.kernel_size))
        self.conv_pqc = qml.qnn.KerasLayer(node, shapes, output_dim= n_qubits)
        
    def build(self, input_shape):
    
        self.iters, _ = convolution_iters(
            input_shape[1:3], self.kernel_size, self.strides, 'valid')
        

    def _convolution(self, input_tensor):

        s = self.strides
        k = self.kernel_size

        conv_out = []
        for i in range(self.iters[0]):
            for j in range(self.iters[1]):
                x = input_tensor[:, i * s[0]:i * s[0] + k[0], j *
                                 s[1]:j * s[1] + k[1]]
                x = tf.keras.layers.Flatten()(x)
                conv_out += [self.conv_pqc(x)]
              
        conv_out = tf.keras.layers.Concatenate(axis=1)(conv_out)
        conv_out = tf.keras.layers.Reshape((self.iters[0], self.iters[1], n_qubits))(conv_out)
        return conv_out

    def call(self, input_tensor):
  
        conv_out = [self._convolution(input_tensor[:, :, :, 0])]
        conv_out = tf.keras.layers.Concatenate(axis=-1)(conv_out)
        return conv_out

In [306]:
rlayer = QConv2D(template = 'Basic')
flayer = tf.keras.layers.Flatten()
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")
model = tf.keras.models.Sequential([rlayer, flayer, clayer_2])

In [307]:
opt = tf.keras.optimizers.Adam(learning_rate=0.2)
model.compile(opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [308]:
dim = data.config()['input_dims']
in_shape = [args.batch_size] + list(dim)

x = tf.random.uniform((in_shape))
y = model(x)
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 QConv2D (QConv2D)           multiple                  18        
                                                                 
 flatten_4872 (Flatten)      multiple                  0         
                                                                 
 dense_21 (Dense)            multiple                  164       
                                                                 
Total params: 182
Trainable params: 182
Non-trainable params: 0
_________________________________________________________________


In [ ]:
fitting = model.fit(data.train_ds,
                    batch_size= args.batch_size,
                   epochs= args.epochs,
                   validation_data=data.val_ds,
                   shuffle=True,
                   workers=4)

Epoch 1/3
12/12 [==============================] - 14s 1s/step - loss: 0.5509 - accuracy: 0.7083 - val_loss: 0.2112 - val_accuracy: 0.9167
Epoch 2/3
12/12 [==============================] - ETA: 0s - loss: 0.1995 - accuracy: 0.9167